In [1]:
import pandas as pd
rna_data = pd.read_csv("gene_data/rna_common_complete.csv")
rna_data = rna_data.sort_values(by=['sn','period']).reset_index(drop=True)

In [2]:
X_og_shape = rna_data.drop(['sn','group','caarms_status','period'],axis=1).values
X_reshaped = X_og_shape.reshape(len(set(rna_data['sn'])), 3, X_og_shape.shape[1])
labels_group = rna_data[rna_data['period'] == 24]['group'].values
labels = [0 if i == 'C' else 1 for i in labels_group]

In [3]:
import torch
import numpy as np
from neucube.utils import SNR
from neucube.utils import interpolate
from neucube.encoder import Delta

ratios = SNR(X_reshaped[:,0,:], labels)
top_idx = torch.argsort(ratios, descending=True)[0:20]
X_reshaped_topidx = X_reshaped[:,:,top_idx]
interpolated_X = interpolate(X_reshaped_topidx, num_points=104)

encoder = Delta(threshold=0.008)
X = encoder.encode_dataset(interpolated_X)
y = torch.tensor(labels)

In [8]:
from neucube.sampler import TemporalBinning
from neucube.utils import SeparationIndex
neuron_parm_dict = { 
    'rs' : {'a': 0.02, 'b': 0.2, 'c': -65, 'd': 8}, 
    'ch' : {'a': 0.02, 'b': 0.55, 'c': -45, 'd': 4},
    'ib' : {'a': 0.06, 'b': 0.55, 'c': -55, 'd': 3},
}

def objective_function(res_, X_stimuli, labels, params):
    a, b, c, d = [torch.tensor(list(map(lambda x: neuron_parm_dict[x][i], params[0]))) for i in ['a', 'b', 'c', 'd']]
    res_.update_parms(a=a, b=b, c=c, d=d)
    out_spikes = res_.simulate(X_stimuli, mem_thr=30, train=False, verbose=True)
    sampler = TemporalBinning(bin_size=10)
    state_vec = sampler.sample(out_spikes)
    return SeparationIndex(state_vec, labels).item()

In [5]:
from neucube import IzhReservoir
izh_res = IzhReservoir(inputs=X.shape[2], c=0.7, l=0.18, input_conn_prob=0.85)
init_n_type = np.random.choice(['rs','ch','ib'], izh_res.n_neurons, replace=True)
izh_res.set_exc_parms(a=0.06, b=0.55, c=-55, d=3)
izh_res.set_inh_parms(a=0.01, b=0.2, c=-65, d=8)

In [6]:
a, b, c, d = [torch.tensor(list(map(lambda x: neuron_parm_dict[x][i], init_n_type))) for i in ['a', 'b', 'c', 'd']]

In [11]:
import nevergrad as ng
import numpy as np
from functools import partial

parametrization = ng.p.Tuple(
    ng.p.Choice(['rs','ch','ib'], repetitions=izh_res.n_neurons)
)

optimizer = ng.optimizers.NoisyDiscreteOnePlusOne(parametrization=parametrization, budget=10)
partial_objective_function = partial(objective_function, res_=izh_res, X_stimuli=X, labels=y)

def print_progress(optimizer):
    for i in range(optimizer.budget):
        x = optimizer.ask()
        loss = -partial_objective_function(params=x.value)
        optimizer.tell(x, loss)
        if (i + 1) % 2 == 0:
            print(f"Iteration {i + 1}/{optimizer.budget}, Current loss: {loss}")

print_progress(optimizer)
recommendation = optimizer.provide_recommendation()
optimal_parms = recommendation.value

print("optimal neuron types:", optimal_parms[0:10])

optimal neuron types: (('ch', 'ch', 'ib', 'ib', 'ch', 'ib', 'ib', 'rs', 'ib', 'ch', 'ch', 'ib', 'rs', 'ch', 'rs', 'ib', 'ib', 'ch', 'rs', 'ib', 'ib', 'rs', 'ch', 'ch', 'rs', 'ib', 'ch', 'rs', 'ib', 'rs', 'rs', 'rs', 'ib', 'ch', 'ib', 'ch', 'ch', 'rs', 'rs', 'ib', 'ch', 'ch', 'rs', 'rs', 'ch', 'ib', 'ch', 'rs', 'ib', 'ch', 'ch', 'ib', 'rs', 'ib', 'rs', 'rs', 'ib', 'ib', 'ib', 'ch', 'ib', 'rs', 'rs', 'ib', 'ib', 'ib', 'rs', 'ch', 'ib', 'rs', 'rs', 'rs', 'ch', 'ch', 'ch', 'rs', 'ib', 'rs', 'ch', 'ib', 'rs', 'ch', 'ch', 'rs', 'ch', 'rs', 'ib', 'ib', 'ch', 'ch', 'ch', 'rs', 'ib', 'rs', 'ib', 'rs', 'rs', 'rs', 'rs', 'ib', 'rs', 'ch', 'rs', 'ib', 'ib', 'ch', 'ch', 'ib', 'ch', 'ib', 'ib', 'rs', 'ib', 'ch', 'rs', 'rs', 'rs', 'rs', 'ib', 'ib', 'rs', 'ib', 'rs', 'ib', 'rs', 'ch', 'ch', 'ch', 'ib', 'ib', 'ch', 'ib', 'rs', 'rs', 'ib', 'rs', 'rs', 'ib', 'rs', 'ib', 'ib', 'ch', 'ib', 'ch', 'ib', 'rs', 'ib', 'ib', 'rs', 'rs', 'rs', 'rs', 'ch', 'ch', 'ch', 'rs', 'ib', 'rs', 'ib', 'rs', 'ib', 'rs', 'ch'

In [12]:
optimal_a, optimal_b, optimal_c, optimal_d = [
    torch.tensor(list(map(lambda x: neuron_parm_dict[x][i], optimal_parms[0]))) for i in ['a', 'b', 'c', 'd']]
izh_res.update_parms(a=optimal_a, b=optimal_b, c=optimal_c, d=optimal_d)
opt_spike = izh_res.simulate(X, mem_thr=30, train=False, verbose=True)

100%|██████████| 115/115 [00:31<00:00,  3.69it/s]


In [13]:
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from neucube.sampler import TemporalBinning
from neucube.utils import SeparationIndex

sampler = TemporalBinning(bin_size=10)
opt_state_vec = sampler.sample(opt_spike)
print(opt_state_vec.shape)

num_folds = 10
kf = KFold(n_splits=num_folds)

true_labels = []
predicted_labels = []

for train_index, test_index in tqdm(kf.split(opt_state_vec)):
    X_train_fold, X_test_fold = opt_state_vec[train_index], opt_state_vec[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    svm = SVC(kernel='linear', C=2)  # You can specify different kernels ('linear', 'poly', 'rbf', etc.)
    svm.fit(X_train_fold, y_train_fold)
    y_pred = svm.predict(X_test_fold)
    true_labels.extend(y_test_fold)
    predicted_labels.extend(y_pred)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print("10-Fold Cross-Validation Accuracy:", accuracy)
print(confusion_matrix(true_labels, predicted_labels))
print("separation:", SeparationIndex(opt_state_vec, y))

torch.Size([115, 11000])


10it [00:01,  9.65it/s]

10-Fold Cross-Validation Accuracy: 0.8260869565217391
[[52 12]
 [ 8 43]]
separation: tensor(0.0114)
